In [1]:
'''
Created on Sep 9, 2017

@author: inayat
'''

# import the required  packages
from imutils.video import WebcamVideoStream
#from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2

import dlib

from trackers.camshifttracker import CAMShiftTracker
	
if __name__ == '__main__':
	print("[info] starting to read a webcam ...")
	
	# Video File
	vid_path = './resources/maze/with_obstacle.mp4'
	# vid_path = './resources/videos/boy-walking.mp4'

	# Set Video Source (Camera / Video File)
	# cap = cv2.VideoCapture(0) # Camera -> use index 0 for default/builtin camera
	capWebCam = cv2.VideoCapture(vid_path) # Video File
	# capWebCam = WebcamVideoStream(1).start()
	# time.sleep(1.0)
	
	ret, roiFrame = capWebCam.read()

	cv2.imshow("Select ROI", roiFrame)
	roi = cv2.selectROI("Select ROI", roiFrame, fromCenter=False, showCrosshair=True)

	# initialize dlib face detector
	
	# frontFaceDetector = dlib.get_frontal_face_detector() 
	
	
	# meanShift tracker
	
	camShifTracker = None
	
	curWindow = None
	
	# start the frame per second  (FPS) counter
	#fps = FPS2().start() 
	
	boolDetectinfirsFrameOnly = True
	
	# loop over the frames obtained from the webcam
	while True:
		# grab each frame from the threaded  stream,
		# resize
		# it, and convert it to grayscale (while still retaining 3
		# channels)
		isFrameFound, frame1 = capWebCam.read()

		if not isFrameFound:
			print("No Frame")
			break

		# frame = cv2.flip(frame1,1)
		frame = frame1 #cv2.flip(frame1,1)
		
		#frame = imutils.resize(frame, width=450)
		#frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		#frame = np.dstack([frame, frame, frame])
		
		# display the size of the queue on the frame
		#cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
		#            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
		
		
		if boolDetectinfirsFrameOnly:
			
			# start the frame per second  (FPS) counter
			# bbox = [(16, 157), (86, 238)]
			bbox = [(roi[0], roi[1]), (roi[2], roi[3])]
			# bbox = (roi[0], roi[1]), (roi[2], roi[3])
			# print(bbox)
						
			# convert dlib rect to opencv rect
			left = min(bbox[0][0], bbox[1][0])
			top = min(bbox[0][1], bbox[1][1])
			right = max(bbox[0][0], bbox[1][0])
			bottom = max(bbox[0][1], bbox[1][1])

			print(left)
			
			# curWindow = (int(bbox.left()), int(bbox.top()), int(bbox.right() - bbox.left()),
			# 			 int(bbox.bottom() - bbox.top()) )
			curWindow = (int(left), int(top), int(right - left),
						 int(bottom - top) )
			
			print(curWindow)
			
			# intialize the CAMShift Tracker
			camShifTracker = CAMShiftTracker(curWindow, frame)
			
			boolDetectinfirsFrameOnly = False
			
			continue 
		
		camShifTracker.computeNewWindow(frame)
		
		x,y, w, h = camShifTracker.getCurWindow()
		
		bkprojectImage = camShifTracker.getBackProjectedImage(frame)
		
		cv2.imshow("CAMShift Face in Back Project Image", bkprojectImage)
		
		# display the current window 
		cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2, cv2.LINE_AA)
		
		rotatedWindow = camShifTracker.getRotatedWindow()
		#display rotated window
		cv2.polylines(frame, [rotatedWindow], True, (0,255,0), 2, cv2.LINE_AA)
		   
		# cv2.putText(frame, "FPS: {:.2f}".format(fps.fps()),
		#             (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
		
		# show the frame and update the FPS counter
		cv2.imshow("CAMShift Face Tracking", frame)
		
		if cv2.waitKey(10) & 0xFF == ord('q'):
			break
		
	# do a bit of cleanup
	capWebCam.release()
	cv2.destroyAllWindows()
	
	

[info] starting to read a webcam ...
60


IndexError: tuple index out of range

: 